## Problem 3: How many people live in the dominance area of each shopping center? 

Find out how many people live under the dominance area of each shopping center. You should be able to do this based on the output from problem 2, lesson materials from weeks 2, 3 and 4 and these additional hints:

- You can read in the population data from the HSY WFS service: https://kartta.hsy.fi/geoserver/wfs (check more hints from the lesson materials)
- Aggregate your dominance areas from problem 2 into a unified geometries using [`dissolve()`](http://geopandas.org/aggregation_with_dissolve.html#dissolve-example) -function in geopandas before joining with the population data.
- Remember to check the crs of the input data.
- Join information between the population grid and the dominance areas -layer using `intersect` as the condition in the spatial join.

You can freely organize your code into the code cells below.



In [18]:
from pyproj import CRS
import geopandas as gpd
import requests
import geojson
import os

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format). 
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))
pop.crs = CRS.from_epsg(3879).to_wkt()
pop = pop[['asukkaita', 'geometry']]
pop.rename(columns={'asukkaita':'population'}, inplace=True)

In [19]:
grid = gpd.read_file(os.path.join('data','grid.shp'))

In [20]:
grid = grid.dissolve(by='dominant_s')

In [21]:
#making sure both have the same crs
pop = pop.to_crs(grid.crs)

In [27]:
join_data = grid.sjoin(pop, predicate='intersects').reset_index()

In [31]:
pop_dominance = join_data.groupby('dominant_s').population.sum()
for shopping_center, population in pop_dominance.iteritems():
    print(f'{population} live under the dominance area of {shopping_center} shopping center')


223770 live under the dominance area of Dixi shopping center
259697 live under the dominance area of Forum shopping center
201205 live under the dominance area of IsoOmena shopping center
212502 live under the dominance area of Itis shopping center
79573 live under the dominance area of Jumbo shopping center
231630 live under the dominance area of Myyrmanni shopping center
87856 live under the dominance area of Ruoholahti shopping center


Remember to print the answers in this notebook :)

### Extra bonus task

Repeat problem 2 and 3 for car accessibility! No extra points available for this, but you can for example start thinking if you want to make the final assignment related to this topic :)